In [ ]:
# --- CELL 1: SETUP & DOWNLOAD ---
!pip install opencv-python matplotlib tensorflow

import os
import zipfile
import gdown

# 1. Download Dataset from Your Drive
file_id = '1aVVieDaek7T7ouia1VqVgrbosTP34KGr'
output_zip = 'dataset.zip'

if not os.path.exists(output_zip):
    print(" Downloading dataset")
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output_zip, quiet=False)
else:
    print(" Dataset already downloaded.")

# 2. Extract Dataset
extract_path = 'My_Dataset'
if not os.path.exists(extract_path):
    print(" Extracting files...")
    with zipfile.ZipFile(output_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(" Extraction Complete!")
else:
    print(" Files already extracted.")

Downloading...
From (original): https://drive.google.com/uc?id=1aVVieDaek7T7ouia1VqVgrbosTP34KGr
From (redirected): https://drive.google.com/uc?id=1aVVieDaek7T7ouia1VqVgrbosTP34KGr&confirm=t&uuid=ec28188e-b8ef-4593-9a81-5316fe06ff62
To: /content/dataset.zip
100%|██████████| 1.65G/1.65G [00:29<00:00, 56.2MB/s]


 Extracting files...
 Extraction Complete!


In [ ]:
#  CELL 2: PROCESSING PIPELINE
import cv2
import numpy as np

class ThermalPipeline:
    def __init__(self, target_size=(64, 64), seq_len=10):
        self.target_size = target_size
        self.seq_len = seq_len

    def process_single_image(self, image_path):

        # 1. Acquisition
        if not os.path.exists(image_path):
            print(f"Error: {image_path} not found")
            return None
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # 2. Preprocessing (Resize & Normalize)
        img = cv2.resize(img, self.target_size)
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=-1)  # Adding channel (H, W, 1)

        # 3. Representation (Repeat to make sequence)
        # We repeat the image 'seq_len' times to simulate time
        sequence = np.repeat(img[np.newaxis, ...], self.seq_len, axis=0)

        return sequence # Shape: (Seq_Len, H, W, 1)

In [ ]:
# CELL 3: LOAD TRAINING DATA
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Configuration
pipeline = ThermalPipeline(target_size=(64, 64), seq_len=10)
data_dir = "My_Dataset"


categories = ["Non-stess", "Stress"]

data = []
labels = []

print(" Loading and processing images... this may take a moment.")

# Walk through folders
for label_idx, category in enumerate(categories):
    folder_path = None
    # Look for the folder ensuring case-insensitive matching just in case
    for root, dirs, files in os.walk(data_dir):
        # We look for the folder name in the directory list
        if category in dirs:
            folder_path = os.path.join(root, category)
            break

    if folder_path:
        print(f" Processing folder: {category}")
        file_count = 0
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(folder_path, filename)

                # Process the image into a sequence
                sequence = pipeline.process_single_image(img_path)

                if sequence is not None:
                    data.append(sequence)
                    labels.append(label_idx)
                    file_count += 1
        print(f"   -> Loaded {file_count} images from {category}")
    else:
        print(f"   ERROR: Still cannot find folder '{category}'")
        print(f"   Debug: Folders found in {data_dir} are: {os.listdir(data_dir)}")

# Convert to Numpy Arrays
if len(data) > 0:
    X = np.array(data)
    y = np.array(labels)

    # Split into Train and Test sets (80% Train, 20% Test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"\n Data Ready!")
    print(f"Training Data Shape: {X_train.shape} (Samples, Time, Height, Width, Channels)")
    print(f"Testing Data Shape: {X_test.shape}")
else:
    print("\n CRITICAL FAILURE: No data was loaded. Please check folder names again.")

 Loading and processing images... this may take a moment.
 Processing folder: Non-stess
   -> Loaded 1556 images from Non-stess
 Processing folder: Stress
   -> Loaded 1603 images from Stress

 Data Ready!
Training Data Shape: (2527, 10, 64, 64, 1) (Samples, Time, Height, Width, Channels)
Testing Data Shape: (632, 10, 64, 64, 1)
